In [2]:
#Import Dependencies
import pandas as pd
from datetime import date
from pathlib import Path

In [3]:
#Set Paths for Data CSV's
clinp=Path("Resources/WHO_NREVSS_Clinical_Labs.csv")
pubp=Path("Resources/WHO_NREVSS_Public_Health_Labs.csv")
comp=Path("Resources/WHO_NREVSS_Combined_prior_to_2015_16.csv")

In [54]:
#Read Data from the CSV's
clinical = pd.read_csv(clinp,header=1)
public = pd.read_csv(pubp, header=1)
combo2016= pd.read_csv(comp, header=1)


public.head()


,REGION TYPE,REGION,SEASON_DESCRIPTION,TOTAL SPECIMENS,A (2009 H1N1),A (H3),A (Subtyping not Performed),B,BVic,BYam,H3N2v,A (H5)
0,States,Alabama,Season 2015-16,256,59,16,1,2,2,2,0,0
1,States,Alaska,Season 2015-16,4691,607,98,0,231,1,2,0,0
2,States,Arizona,Season 2015-16,2110,762,580,0,13,58,399,0,0
3,States,Arkansas,Season 2015-16,128,20,8,0,1,12,0,0,0
4,States,California,Season 2015-16,12820,1462,854,35,775,309,803,0,0


In [ ]:
#Isolate the data taken during flu season (October-May)
clinical["Season"]=clinical[["YEAR","WEEK"]].apply(lambda row: date.fromisocalendar(int(row['YEAR']), int(row['WEEK']),1).month,axis=1)
clinical=clinical.drop(clinical[(clinical.Season<10)&(clinical.Season>5)].index)

#Drop X only rows
clinical=clinical[clinical["TOTAL SPECIMENS"]!="X"]

#Convert date data into "Season year1-year2" format present in the Public dataset
clinical.loc[(clinical["Season"]<10),"YEAR"]=clinical["YEAR"].apply(lambda x: x-1)
clinical["YEAR_2"]=clinical["YEAR"].apply(lambda x: str(x+1)[2:4])
clinical["Season"]=clinical[["YEAR","YEAR_2"]].apply(lambda row: "Season "+str(row["YEAR"])+"-"+row["YEAR_2"],axis=1)

#Convert Numerical Data into integers
clinical=clinical.astype({"TOTAL SPECIMENS":"int32","TOTAL A":"int32","TOTAL B":"int32"})

#Groupby Season and Region, summing the weekly data into season data
clinical=clinical[["Season","REGION","TOTAL SPECIMENS","TOTAL A","TOTAL B"]].groupby(["Season","REGION"]).sum(numeric_only=1)
print(clinical)



                              TOTAL SPECIMENS  TOTAL A  TOTAL B
Season         REGION                                          
Season 2015-16 Alabama                   8727      382       68
               Alaska                     306       57       18
               Arizona                   3460      285       56
               Arkansas                  2022       47      114
               California               43964     2755     2020
...                                       ...      ...      ...
Season 2025-26 Virginia                  1388      247        2
               Washington                3925      562       29
               West Virginia              733       94        1
               Wisconsin                16193     2479       23
               Wyoming                     21        5        0

[494 rows x 3 columns]


In [ ]:
#Replace isolated X's with 0's
combo2016=combo2016[combo2016["TOTAL SPECIMENS"]!="X"]
combo2016=combo2016[(combo2016["A (2009 H1N1)"]=="X")|
          (combo2016["A (H1)"]=="X")|
          (combo2016["A (H3)"]=="X")|
          (combo2016["A (Subtyping not Performed)"]=="X")|
          (combo2016["A (Unable to Subtype)"]=="X")|
          (combo2016["B"]=="X")|
          (combo2016["H3N2v"]=="X")|
          (combo2016["A (H5)"]=="X")]
combo2016[combo2016["A (H5)"]=="X"].head()

,REGION TYPE,REGION,YEAR,WEEK,TOTAL SPECIMENS,PERCENT POSITIVE,A (2009 H1N1),A (H1),A (H3),A (Subtyping not Performed),A (Unable to Subtype),B,H3N2v,A (H5)
0,States,Alabama,2010,40,54,0,0,0,0,0,0,0,0,X
1,States,Alaska,2010,40,40,0,0,0,0,0,0,0,0,X
2,States,Arizona,2010,40,40,2.5,0,0,1,0,0,0,0,X
3,States,Arkansas,2010,40,15,0,0,0,0,0,0,0,0,X
4,States,California,2010,40,183,3.28,2,0,3,0,0,1,0,X
